# TreePlantr
# 🌳 To Tree or Not To Tree: GIS Planting Tool (v1.0 Beta)

This is a Python-based tool to help estimate where trees can realistically be planted in a city — taking into account not just land cover, but also legal/code restrictions and cultural considerations (like sports fields or sidewalks).

This version is **in beta** — it's fully scripted, functional, and outputs clean shapefiles and a Leaflet map, but still being tested for edge cases and real-world quirks.

---

## 🔍 What it Does

- Reclassifies land cover raster into 4 categories:  
  `Plantable`, `Not Plantable`, `Water`, and `Tree Canopy`
- Accepts user-defined **code-based** and **cultural** restriction layers with buffer distances
- Buffers restriction areas and subtracts them from plantable zones
- Flags “special case” locations (like golf courses or cemeteries)
- Optionally intersects plantable areas with **ownership** layers:
  - Right-of-way
  - Public parcels
  - Private parcels
- Saves all outputs to `_exports/` folder
- Creates a simple **interactive Leaflet map** (`planting_map.html`)
- Writes a readable text summary of stats — no fancy charts

---

## 🛠 Requirements

- Python 3.9+
- Libraries:  
  `geopandas`, `rasterio`, `folium`, `shapely`, `numpy`, `pandas`

Install with:
```bash
pip install geopandas rasterio folium shapely numpy pandas

In [ ]:
# 🌳 To Tree or Not To Tree: GIS Analysis Notebook
# =====================================================
# This notebook estimates plantable land in a city by filtering out land
# based on land cover, code/cultural restrictions, and optional ownership.
#
# ⚙️ REQUIREMENTS: rasterio, geopandas, shapely, folium, numpy
# 📁 OUTPUTS: All shapefiles saved to ./_exports/

# ============================================
# 1️⃣ SETUP
# ============================================
import os
import geopandas as gpd
import rasterio
from rasterio.features import shapes
from shapely.geometry import shape
from shapely.ops import unary_union
import numpy as np
import folium
import pandas as pd

EXPORT_DIR = "_exports"
os.makedirs(EXPORT_DIR, exist_ok=True)

# ‼️ STEP 1: DEFINE LAND COVER CLASSES BASED ON RASTER VALUES
landcover_definitions = {
    "plantable": [1, 2, 8, 9, 10],       # Grass, Bare Soil, Ag, Wetlands, Forest Wetland
    "not_plantable": [3, 4, 12],        # Buildings, Roads, Extraction
    "water": [5, 11],                   # Lakes, Rivers
    "tree_cover": [6, 7]                # Tree canopy
}

# ‼️ STEP 2: DEFINE RESTRICTION LAYERS
special_cases = [
    {"name": "Wetlands", "path": "data/special/wetlands.shp"},
    {"name": "Airfields", "path": "data/special/airfields.shp"},
    {"name": "Golf Course", "path": "data/special/golf_courses.shp"},
    {"name": "Graveyard", "path": "data/special/graveyards.shp"}
]

culture_restrictions = [
    {"name": "Baseball Field", "path": "data/culture/baseball.shp", "buffer": 15},
    {"name": "Sidewalk", "path": "data/culture/sidewalks.shp", "buffer": 4},
    {"name": "Rail Corridor", "path": "data/culture/rail.shp", "buffer": 25}
]

code_restrictions = [
    {"name": "Traffic Signals", "path": "data/code/signals.shp", "buffer": 30},
    {"name": "Street Lights", "path": "data/code/lights.shp", "buffer": 15},
    {"name": "Fire Hydrants", "path": "data/code/hydrants.shp", "buffer": 10}
]

# ‼️ STEP 3: OPTIONAL OWNERSHIP LAYERS
ownership_layers = {
    "right_of_way": "data/ownership/ROW.shp",
    "public_parcels": "data/ownership/public.shp",
    "private_parcels": "data/ownership/private.shp"
}

# ============================================
# 2️⃣ LAND COVER CLASSIFICATION
# ============================================

# ‼️ EDIT THIS TO POINT TO YOUR RASTER FILE
RASTER_PATH = "data/landcover_2019.tif"

# Turn classified raster into vector polygons
landcover_shapes = []
with rasterio.open(RASTER_PATH) as src:
    raster = src.read(1)
    mask = raster != src.nodata
    results = shapes(raster, mask=mask, transform=src.transform)
    for geom, val in results:
        for class_name, values in landcover_definitions.items():
            if val in values:
                landcover_shapes.append({"geometry": shape(geom), "value": val, "class": class_name})

landcover_gdf = gpd.GeoDataFrame(landcover_shapes, crs=src.crs)
landcover_gdf.to_file(f"{EXPORT_DIR}/landcover_classified.shp")

# Save plantable layer for analysis
plantable_gdf = landcover_gdf[landcover_gdf["class"] == "plantable"].copy()
plantable_gdf.to_file(f"{EXPORT_DIR}/plantable_unbuffered.shp")

# ============================================
# 3️⃣ SPECIAL CASES (NO BUFFER)
# ============================================

special_gdfs = []
for case in special_cases:
    gdf = gpd.read_file(case["path"])
    gdf["name"] = case["name"]
    special_gdfs.append(gdf)

all_special = gpd.GeoDataFrame(pd.concat(special_gdfs, ignore_index=True), crs=gdf.crs)
all_special.to_file(f"{EXPORT_DIR}/special_cases.shp")

# ============================================
# 4️⃣ BUFFER RESTRICTIONS (CODE + CULTURE)
# ============================================

def build_buffer(layers):
    buffers = []
    for entry in layers:
        gdf = gpd.read_file(entry["path"])
        gdf = gdf.to_crs(plantable_gdf.crs)
        gdf["name"] = entry["name"]
        gdf["buffer_dist"] = entry["buffer"]
        gdf["geometry"] = gdf.buffer(entry["buffer"])
        buffers.append(gdf)
    return gpd.GeoDataFrame(pd.concat(buffers, ignore_index=True), crs=plantable_gdf.crs)

code_gdf = build_buffer(code_restrictions)
code_gdf.to_file(f"{EXPORT_DIR}/code_buffer.shp")

culture_gdf = build_buffer(culture_restrictions)
culture_gdf.to_file(f"{EXPORT_DIR}/culture_buffer.shp")

# Merge total restriction buffer
total_buffer = gpd.overlay(code_gdf, culture_gdf, how='union')
total_buffer = total_buffer.dissolve()  # dissolve to single multipart polygon

# Save
total_buffer.to_file(f"{EXPORT_DIR}/total_buffer.shp")

# ============================================
# 5️⃣ ERASE BUFFER FROM PLANTABLE AREAS
# ============================================

plantable_cleaned = gpd.overlay(plantable_gdf, total_buffer, how='difference')
plantable_cleaned.to_file(f"{EXPORT_DIR}/plantable_final.shp")

# ============================================
# 6️⃣ OPTIONAL: OWNERSHIP ANALYSIS (NOW INCLUDED)
# ============================================
ownership_results = []

for label, path in ownership_layers.items():
    try:
        layer = gpd.read_file(path).to_crs(plantable_cleaned.crs)
        clipped = gpd.overlay(plantable_cleaned, layer, how="intersection")
        area = clipped.area.sum() / 4046.86  # acres
        ownership_results.append((label, area))
        clipped.to_file(f"{EXPORT_DIR}/plantable_in_{label}.shp")
    except Exception as e:
        ownership_results.append((label, f"Error: {e}"))

# ============================================
# 7️⃣ MAP RESULTS IN FOLIUM
# ============================================

m = folium.Map(location=[44.95, -93.09], zoom_start=12, tiles="CartoDB positron")

for name, file in [
    ("Landcover Classified", "landcover_classified.shp"),
    ("Special Cases", "special_cases.shp"),
    ("Code Buffer", "code_buffer.shp"),
    ("Culture Buffer", "culture_buffer.shp"),
    ("Total Buffer", "total_buffer.shp"),
    ("Plantable Final", "plantable_final.shp")
]:
    gdf = gpd.read_file(f"{EXPORT_DIR}/{file}")
    gdf = gdf.to_crs(epsg=4326)
    folium.GeoJson(gdf, name=name).add_to(m)

folium.LayerControl().add_to(m)
m.save(f"{EXPORT_DIR}/planting_map.html")

# ============================================
# 8️⃣ SUMMARY STATS
# ============================================

plantable_acres = plantable_gdf.area.sum() / 4046.86
plantable_cleaned_acres = plantable_cleaned.area.sum() / 4046.86
lost_to_buffers = plantable_acres - plantable_cleaned_acres

with open(f"{EXPORT_DIR}/summary.txt", "w") as f:
    f.write(f"Original plantable area: {plantable_acres:,.2f} acres\n")
    f.write(f"Plantable area after buffers: {plantable_cleaned_acres:,.2f} acres\n")
    f.write(f"Lost to buffer restrictions: {lost_to_buffers:,.2f} acres\n\n")
    f.write("Ownership breakdown:\n")
    for label, value in ownership_results:
        f.write(f" - {label}: {value}\n")
